In [1]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
from textblob import TextBlob

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
import csv
from nltk.sentiment import SentimentIntensityAnalyzer

def analyze_sentiment(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)["compound"]
    if sentiment > 0:
        return "Positive"
    elif sentiment < 0:
        return "Negative"
    else:
        return "Neutral"

sentiments = []
timestamps = []
sentiment_changes = []
likes = []
retweets = []

with open('chatgpt_daily_tweets.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    headers = next(reader)  # Read the header row
    tweet_index = headers.index("text")  # Get the index of the "text" column
    timestamp_index = headers.index("tweet_extracted")  # Get the index of the "tweet_extracted" column
    likes_index = headers.index("like_count")  # Get the index of the "like_count" column
    retweets_index = headers.index("retweet_count")  # Get the index of the "retweet_count" column

    prev_sentiment = None
    count = 0
    for row in reader:
        if count >= 3000:
           break
        if len(row) >= tweet_index + 1 and len(row) >= timestamp_index + 1:
            tweet = row[tweet_index]
            timestamp = row[timestamp_index]
            sentiment = analyze_sentiment(tweet)
            sentiments.append(sentiment)
            timestamps.append(timestamp)

            if prev_sentiment is not None:
                sentiment_change = sentiment != prev_sentiment
                sentiment_changes.append(sentiment_change)
            prev_sentiment = sentiment

            like_count = 0
            retweet_count = 0

            if len(row) >= likes_index + 1 and row[likes_index]:
                try:
                    like_count = float(row[likes_index])
                except ValueError:
                    pass

            if len(row) >= retweets_index + 1 and row[retweets_index]:
                try:
                    retweet_count = float(row[retweets_index])
                except ValueError:
                    pass

            likes.append(like_count)
            retweets.append(retweet_count)

            count += 1

total_count = len(sentiments)
positive_count = sentiments.count("Positive")
negative_count = sentiments.count("Negative")
neutral_count = sentiments.count("Neutral")

positive_percentage = (positive_count / total_count) * 100
negative_percentage = (negative_count / total_count) * 100
neutral_percentage = (neutral_count / total_count) * 100

print("Συγκεντρωτικά συναισθήματα:")
print("Positive: {} ({}%)".format(positive_count, positive_percentage))
print("Negative: {} ({}%)".format(negative_count, negative_percentage))
print("Neutral: {} ({}%)".format(neutral_count, neutral_percentage))

sentiment_change_count = sentiment_changes.count(True)
sentiment_change_percentage = (sentiment_change_count / total_count) * 100

print("\nΜεταβολή συναισθήματος:")
print("Αριθμός μεταβολών: {}".format(sentiment_change_count))
print("Ποσοστό μεταβολών: {:.2f}%".format(sentiment_change_percentage))

average_likes = sum(likes) / len(likes)
average_retweets = sum(retweets) / len(retweets)

print("\nΜέσος όρος Likes: {:.2f}".format(average_likes))
print("Μέσος όρος Retweets: {:.2f}".format(average_retweets))

print("\nΜεταβολή συναισθήματος στον χρόνο:")
for i in range(1, len(timestamps)):
    if sentiment_changes[i - 1]:
        print("Μεταβολή συναισθήματος από {} σε {} στις {}".format(sentiments[i - 1], sentiments[i], timestamps[i]))


Συγκεντρωτικά συναισθήματα:
Positive: 668 (22.26666666666667%)
Negative: 529 (17.633333333333333%)
Neutral: 1803 (60.099999999999994%)

Μεταβολή συναισθήματος:
Αριθμός μεταβολών: 1680
Ποσοστό μεταβολών: 56.00%

Μέσος όρος Likes: 2.28
Μέσος όρος Retweets: 1235.39

Μεταβολή συναισθήματος στον χρόνο:
Μεταβολή συναισθήματος από Neutral σε Positive στις 2023-04-08 01:06:52.504868
Μεταβολή συναισθήματος από Positive σε Negative στις 2023-04-08 01:06:50.638068
Μεταβολή συναισθήματος από Negative σε Positive στις 2023-04-08 01:07:00.375167
Μεταβολή συναισθήματος από Positive σε Neutral στις 2023-04-08 01:06:49.813797
Μεταβολή συναισθήματος από Neutral σε Positive στις 2023-04-08 01:07:01.437161
Μεταβολή συναισθήματος από Positive σε Negative στις 2023-04-08 01:07:10.575474
Μεταβολή συναισθήματος από Negative σε Neutral στις 2023-04-08 01:07:05.916322
Μεταβολή συναισθήματος από Neutral σε Positive στις 2023-04-08 01:06:59.379927
Μεταβολή συναισθήματος από Positive σε Negative στις 2023-04-08 01